<a href="https://colab.research.google.com/github/PacktPublishing/Modern-Computer-Vision-with-PyTorch-2E/blob/main/Chapter04/Visualizing_the_filters'_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [3]:
# !mkdir _data
# !cd _data
# !wget https://www.dropbox.com/s/5jh4hpuk2gcxaaq/all.zip
# !unzip all.zip

mkdir: _data: File exists
--2025-01-07 10:20:49--  https://www.dropbox.com/s/5jh4hpuk2gcxaaq/all.zip
正在解析主机 www.dropbox.com (www.dropbox.com)... 2a03:2880:f12c:183:face:b00c:0:25de, 209.95.56.60
正在连接 www.dropbox.com (www.dropbox.com)|2a03:2880:f12c:183:face:b00c:0:25de|:443... ^C
unzip:  cannot find or open all.zip, all.zip.zip or all.zip.ZIP.


In [ ]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import SGD, Adam

device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision import datasets
import numpy as np, cv2
import matplotlib.pyplot as plt
from glob import glob

%matplotlib inline
from imgaug import augmenters as iaa

In [ ]:
tfm = iaa.Sequential(iaa.Resize(28))


class XO(Dataset):
    def __init__(self, folder):
        self.files = glob(folder)

    def __len__(self):
        return len(self.files)

    def __getitem__(self, ix):
        f = self.files[ix]
        im = tfm.augment_image(cv2.imread(f)[:, :, 0])
        im = im[None]
        cl = f.split("/")[-1].split("@")[0] == "x"
        return torch.tensor(1 - im / 255).to(device).float(), torch.tensor(
            [cl]
        ).float().to(device)

In [ ]:
data = XO("/content/all/*")

In [ ]:
R, C = 7, 7
fig, ax = plt.subplots(R, C, figsize=(5, 5))
for label_class, plot_row in enumerate(ax):
    for plot_cell in plot_row:
        plot_cell.grid(False)
        plot_cell.axis("off")
        ix = np.random.choice(1000)
        im, label = data[ix]
        print()
        plot_cell.imshow(im[0].cpu(), cmap="gray")
plt.tight_layout()

In [ ]:
from torch.optim import SGD, Adam


def get_model():
    model = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3),
        nn.MaxPool2d(2),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(3200, 256),
        nn.ReLU(),
        nn.Linear(256, 1),
        nn.Sigmoid(),
    ).to(device)

    loss_fn = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

In [ ]:
def train_batch(x, y, model, opt, loss_fn):
    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    return batch_loss.item()


def accuracy(x, y, model):
    with torch.no_grad():
        prediction = model(x)
    max_values, argmaxes = prediction.max(-1)
    is_correct = argmaxes == y
    return is_correct.cpu().numpy().tolist()

In [ ]:
trn_dl = DataLoader(XO("/content/all/*"), batch_size=32, drop_last=True)

In [ ]:
def val_loss(x, y, model):
    with torch.no_grad():
        prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [ ]:
model, loss_fn, optimizer = get_model()

In [ ]:
from torchsummary import summary

model, loss_fn, optimizer = get_model()
summary(model, input_size=(1, 28, 28))

In [ ]:
for epoch in range(5):
    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)

In [ ]:
im, c = trn_dl.dataset[2]
plt.imshow(im[0].cpu())
plt.show()

In [ ]:
first_layer = nn.Sequential(*list(model.children())[:1])
intermediate_output = first_layer(im[None])[0].detach()

In [ ]:
n = 8
fig, ax = plt.subplots(n, n, figsize=(10, 10))
for ix, axis in enumerate(ax.flat):
    axis.set_title("Filter: " + str(ix))
    axis.imshow(intermediate_output[ix].cpu())
plt.tight_layout()
plt.show()

In [ ]:
list(model.children())

In [ ]:
second_layer = nn.Sequential(*list(model.children())[:4])
second_intermediate_output = second_layer(im[None])[0].detach()

In [ ]:
second_intermediate_output.shape

In [ ]:
n = 11
fig, ax = plt.subplots(n, n, figsize=(10, 10))
for ix, axis in enumerate(ax.flat):
    axis.imshow(second_intermediate_output[ix].cpu())
    axis.set_title(str(ix))
plt.tight_layout()
plt.show()

In [ ]:
im.shape

In [ ]:
x, y = next(iter(trn_dl))
x2 = x[y == 0]
len(x2)

In [ ]:
x2 = x2.view(-1, 1, 28, 28)

In [ ]:
first_layer = nn.Sequential(*list(model.children())[:1])
first_layer_output = first_layer(x2).detach()

In [ ]:
n = 4
fig, ax = plt.subplots(n, n, figsize=(10, 10))
for ix, axis in enumerate(ax.flat):
    axis.imshow(first_layer_output[ix, 4, :, :].cpu())
    axis.set_title(str(ix))
plt.tight_layout()
plt.show()

In [ ]:
second_layer = nn.Sequential(*list(model.children())[:4])
second_intermediate_output = second_layer(x2).detach()

In [ ]:
second_intermediate_output.shape

In [ ]:
n = 4
fig, ax = plt.subplots(n, n, figsize=(10, 10))
for ix, axis in enumerate(ax.flat):
    axis.imshow(second_intermediate_output[ix, 34, :, :].cpu())
    axis.set_title(str(ix))
plt.tight_layout()
plt.show()

In [ ]:
len(XO("/content/all/*"))

In [ ]:
custom_dl = DataLoader(XO("/content/all/*"), batch_size=2498, drop_last=True)

In [ ]:
x, y = next(iter(custom_dl))

In [ ]:
x2 = x[y == 0]

In [ ]:
len(x2)

In [ ]:
x2 = x2.view(len(x2), 1, 28, 28)

In [ ]:
flatten_layer = nn.Sequential(*list(model.children())[:7])
flatten_layer_output = flatten_layer(x2).detach()

In [ ]:
flatten_layer_output.shape

In [ ]:
plt.figure(figsize=(100, 10))
plt.imshow(flatten_layer_output.cpu())